In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

In [2]:
engine=create_engine("sqlite:///inventory.db")

In [3]:
def ingest_db(df,table_name,engine):
    df.to_sql(table_name,con=engine,if_exists='replace',index=False)

In [4]:
for file in os.listdir("data"):
    if ".csv" in file:
        df=pd.read_csv("data/"+file)
        print(df.shape)
        ingest_db(df,file[:-4],engine)

(2372474, 16)
(12261, 9)
(12825363, 14)
(5543, 10)


In [5]:
import sqlite3

In [6]:
conn=sqlite3.connect("inventory.db")

In [7]:
tables=pd.read_sql_query("select name from sqlite_master where type='table'",conn)
tables

,name
0,purchases
1,purchase_prices
2,sales
3,vendor_invoice


In [8]:
purchases=pd.read_sql_query("select * from purchases",conn)
purchases.head()

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1


In [9]:
purchase_prices=pd.read_sql_query("select * from purchase_prices",conn)
purchase_prices.head()

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


In [10]:
sales=pd.read_sql_query("select * from sales",conn)
sales.head()

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [11]:
vendor_invoice=pd.read_sql_query("select * from vendor_invoice",conn)
vendor_invoice.head()

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None


In [12]:
purchase_summary=pd.read_sql_query(""" select 
p.Brand,
p.Description,
p.VendorNumber,
p.VendorName,
pp.Price as ActualPrice,
pp.Volume,
p.PurchasePrice,
sum(p.Quantity) as Total_Purchase_Quantity,
sum(p.Dollars) as Total_Purchase_Dollars
from purchases p join purchase_prices pp on p.Brand=pp.Brand
where p.PurchasePrice>0
group by p.Brand,p.Description,p.VendorNumber,p.VendorName,pp.Price,pp.Volume,p.PurchasePrice
order by Total_Purchase_Dollars
""",conn)
purchase_summary

,Brand,Description,VendorNumber,VendorName,ActualPrice,Volume,PurchasePrice,Total_Purchase_Quantity,Total_Purchase_Dollars
0,3065,Three Olives Grape Vodka,7245,PROXIMO SPIRITS INC.,0.99,50,0.71,1,0.71
1,6127,The Club Strawbry Margarita,3960,DIAGEO NORTH AMERICA INC,1.99,200,1.47,1,1.47
2,9123,Deep Eddy Vodka,3924,HEAVEN HILL DISTILLERIES,0.99,50,0.74,2,1.48
3,5683,Dr McGillicuddy's Apple Pie,8004,SAZERAC CO INC,0.49,50,0.39,6,2.34
4,8527,Concannon Glen Ellen Wh Zin,9815,WINE GROUP INC,4.99,750,1.32,2,2.64
...,...,...,...,...,...,...,...,...,...
10687,3545,Ketel One Vodka,3960,DIAGEO NORTH AMERICA INC,29.99,1750,21.89,138109,3023206.01
10688,4261,Capt Morgan Spiced Rum,3960,DIAGEO NORTH AMERICA INC,22.99,1750,16.17,201682,3261197.94
10689,8068,Absolut 80 Proof,17035,PERNOD RICARD USA,24.99,1750,18.24,187407,3418303.68
10690,3405,Tito's Handmade Vodka,4425,MARTIGNETTI COMPANIES,28.99,1750,23.19,164038,3804041.22


In [13]:
sales_summary=pd.read_sql_query(""" select
Brand,
VendorNo,
sum(SalesQuantity)as Total_Sale_Quantity,
sum(SalesDollars)as Total_Sale_Dollars,
sum(SalesPrice) as Total_Sale_Price,
sum(ExciseTax) as Total_Excise_Tax
from sales group by Brand,VendorNo
""",conn)
sales_summary

,Brand,VendorNo,Total_Sale_Quantity,Total_Sale_Dollars,Total_Sale_Price,Total_Excise_Tax
0,58,8320,3446,43341.54,28145.64,2715.12
1,60,12546,1775,18716.25,10720.79,1398.56
2,61,8004,312,4364.88,363.74,245.70
3,62,1128,3125,119863.75,90154.51,2463.18
4,63,1128,2778,112249.22,88553.10,2190.45
...,...,...,...,...,...,...
11267,90090,1703,15,9749.85,4549.93,1.68
11268,90604,1703,76,9119.24,2639.78,8.53
11269,90609,2,24,599.76,449.82,0.52
11270,90609,201359,59,1474.41,1324.47,1.24


In [14]:
freight_summary=pd.read_sql_query("""select
VendorNumber,
sum(Freight) as Freight_Cost
from vendor_invoice group by VendorNumber
""",conn)
freight_summary

,VendorNumber,Freight_Cost
0,2,27.08
1,54,0.48
2,60,367.52
3,105,62.39
4,200,6.19
...,...,...
121,98450,856.02
122,99166,130.09
123,172662,178.34
124,173357,202.50


In [15]:
vendor_sale_summery=pd.read_sql_query(""" 
with purchasesummary as (
select 
p.Brand,
p.Description,
p.VendorNumber,
p.VendorName,
pp.Price as ActualPrice,
pp.Volume,
p.PurchasePrice,
sum(p.Quantity) as Total_Purchase_Quantity,
sum(p.Dollars) as Total_Purchase_Dollars
from purchases p join purchase_prices pp on p.Brand=pp.Brand
where p.PurchasePrice>0
group by p.Brand,p.Description,p.VendorNumber,p.VendorName,pp.Price,pp.Volume,p.PurchasePrice
order by Total_Purchase_Dollars
),

salessummary as (
select
Brand,
VendorNo,
sum(SalesQuantity)as Total_Sale_Quantity,
sum(SalesDollars)as Total_Sale_Dollars,
sum(SalesPrice) as Total_Sale_Price,
sum(ExciseTax) as Total_Excise_Tax
from sales group by Brand,VendorNo

),
freightsummary as (
select
VendorNumber,
sum(Freight) as Freight_Cost
from vendor_invoice group by VendorNumber
)

select 
ps.VendorNumber,
ps.VendorName,
ps.Brand,
ps.Description,
ps.ActualPrice,
ps.PurchasePrice,
ps.Volume,
ps.Total_Purchase_Quantity,
ps.Total_Purchase_Dollars,
ss.Total_Sale_Quantity,
ss.Total_Sale_Dollars,
ss.Total_Sale_Price,
ss.Total_Excise_Tax,
fs.Freight_Cost
from purchasesummary ps left join salessummary ss on ps.Brand=ss.Brand and ps.VendorNumber=ss.VendorNo 
left join freightsummary fs on ps.VendorNumber=fs.VendorNumber order by ps.Total_Purchase_Dollars desc
""",conn)
vendor_sale_summery

,VendorNumber,VendorName,Brand,Description,ActualPrice,PurchasePrice,Volume,Total_Purchase_Quantity,Total_Purchase_Dollars,Total_Sale_Quantity,Total_Sale_Dollars,Total_Sale_Price,Total_Excise_Tax,Freight_Cost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,36.99,26.27,1750,145080,3811251.60,142049.0,5.101920e+06,672819.31,260999.20,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,28.99,23.19,1750,164038,3804041.22,160247.0,4.819073e+06,561512.37,294438.66,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,24.99,18.24,1750,187407,3418303.68,187140.0,4.538121e+06,461140.15,343854.07,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,22.99,16.17,1750,201682,3261197.94,200412.0,4.475973e+06,420050.01,368242.80,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,29.99,21.89,1750,138109,3023206.01,135838.0,4.223108e+06,545778.28,249587.83,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,4.99,1.32,750,2,2.64,5.0,1.595000e+01,10.96,0.55,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.49,0.39,50,6,2.34,134.0,6.566000e+01,1.47,7.04,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.99,0.74,50,2,1.48,2.0,1.980000e+00,0.99,0.10,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.99,1.47,200,1,1.47,72.0,1.432800e+02,77.61,15.12,257032.07


In [16]:
vendor_sale_summery.shape

(10692, 14)

In [17]:
vendor_sale_summery.columns

Index(['VendorNumber', 'VendorName', 'Brand', 'Description', 'ActualPrice',
       'PurchasePrice', 'Volume', 'Total_Purchase_Quantity',
       'Total_Purchase_Dollars', 'Total_Sale_Quantity', 'Total_Sale_Dollars',
       'Total_Sale_Price', 'Total_Excise_Tax', 'Freight_Cost'],
      dtype='object')

In [18]:
vendor_sale_summery.dtypes

VendorNumber                 int64
VendorName                  object
Brand                        int64
Description                 object
ActualPrice                float64
PurchasePrice              float64
Volume                      object
Total_Purchase_Quantity      int64
Total_Purchase_Dollars     float64
Total_Sale_Quantity        float64
Total_Sale_Dollars         float64
Total_Sale_Price           float64
Total_Excise_Tax           float64
Freight_Cost               float64
dtype: object

In [19]:
vendor_sale_summery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   VendorNumber             10692 non-null  int64  
 1   VendorName               10692 non-null  object 
 2   Brand                    10692 non-null  int64  
 3   Description              10692 non-null  object 
 4   ActualPrice              10692 non-null  float64
 5   PurchasePrice            10692 non-null  float64
 6   Volume                   10692 non-null  object 
 7   Total_Purchase_Quantity  10692 non-null  int64  
 8   Total_Purchase_Dollars   10692 non-null  float64
 9   Total_Sale_Quantity      10514 non-null  float64
 10  Total_Sale_Dollars       10514 non-null  float64
 11  Total_Sale_Price         10514 non-null  float64
 12  Total_Excise_Tax         10514 non-null  float64
 13  Freight_Cost             10692 non-null  float64
dtypes: float64(8), int64(3

In [20]:
vendor_sale_summery.isnull().sum()

VendorNumber                 0
VendorName                   0
Brand                        0
Description                  0
ActualPrice                  0
PurchasePrice                0
Volume                       0
Total_Purchase_Quantity      0
Total_Purchase_Dollars       0
Total_Sale_Quantity        178
Total_Sale_Dollars         178
Total_Sale_Price           178
Total_Excise_Tax           178
Freight_Cost                 0
dtype: int64

In [21]:
vendor_sale_summery["VendorName"].unique()

array(['BROWN-FORMAN CORP          ', 'MARTIGNETTI COMPANIES',
       'PERNOD RICARD USA          ', 'DIAGEO NORTH AMERICA INC   ',
       'BACARDI USA INC            ', 'JIM BEAM BRANDS COMPANY    ',
       'MAJESTIC FINE WINES        ', 'ULTRA BEVERAGE COMPANY LLP ',
       'STOLI GROUP,(USA) LLC      ', 'PROXIMO SPIRITS INC.       ',
       'MOET HENNESSY USA INC      ', 'CAMPARI AMERICA            ',
       'SAZERAC CO INC             ', 'CONSTELLATION BRANDS INC   ',
       'M S WALKER INC             ', 'SAZERAC NORTH AMERICA INC. ',
       'PALM BAY INTERNATIONAL INC ', 'REMY COINTREAU USA INC     ',
       'SIDNEY FRANK IMPORTING CO  ', 'E & J GALLO WINERY         ',
       'WILLIAM GRANT & SONS INC   ', 'HEAVEN HILL DISTILLERIES   ',
       'DISARONNO INTERNATIONAL LLC', 'EDRINGTON AMERICAS         ',
       'CASTLE BRANDS CORP.        ', 'SOUTHERN WINE & SPIRITS NE ',
       'STE MICHELLE WINE ESTATES  ', 'TRINCHERO FAMILY ESTATES   ',
       'MHW LTD                    ', 'W

In [22]:
vendor_sale_summery["Description"].unique()

array(['Jack Daniels No 7 Black', "Tito's Handmade Vodka",
       'Absolut 80 Proof', ..., 'Crown Royal Apple',
       'Concannon Glen Ellen Wh Zin', 'The Club Strawbry Margarita'],
      dtype=object)

In [24]:
vendor_sale_summery["Volume"]=vendor_sale_summery["Volume"].astype("float64")
vendor_sale_summery.dtypes

VendorNumber                 int64
VendorName                  object
Brand                        int64
Description                 object
ActualPrice                float64
PurchasePrice              float64
Volume                     float64
Total_Purchase_Quantity      int64
Total_Purchase_Dollars     float64
Total_Sale_Quantity        float64
Total_Sale_Dollars         float64
Total_Sale_Price           float64
Total_Excise_Tax           float64
Freight_Cost               float64
dtype: object

In [25]:
vendor_sale_summery.fillna(0,inplace=True)

In [26]:
vendor_sale_summery.isnull().sum()

VendorNumber               0
VendorName                 0
Brand                      0
Description                0
ActualPrice                0
PurchasePrice              0
Volume                     0
Total_Purchase_Quantity    0
Total_Purchase_Dollars     0
Total_Sale_Quantity        0
Total_Sale_Dollars         0
Total_Sale_Price           0
Total_Excise_Tax           0
Freight_Cost               0
dtype: int64

In [27]:
vendor_sale_summery["VendorName"]=vendor_sale_summery["VendorName"].str.strip()

In [28]:
vendor_sale_summery["VendorName"].unique()

array(['BROWN-FORMAN CORP', 'MARTIGNETTI COMPANIES', 'PERNOD RICARD USA',
       'DIAGEO NORTH AMERICA INC', 'BACARDI USA INC',
       'JIM BEAM BRANDS COMPANY', 'MAJESTIC FINE WINES',
       'ULTRA BEVERAGE COMPANY LLP', 'STOLI GROUP,(USA) LLC',
       'PROXIMO SPIRITS INC.', 'MOET HENNESSY USA INC', 'CAMPARI AMERICA',
       'SAZERAC CO INC', 'CONSTELLATION BRANDS INC', 'M S WALKER INC',
       'SAZERAC NORTH AMERICA INC.', 'PALM BAY INTERNATIONAL INC',
       'REMY COINTREAU USA INC', 'SIDNEY FRANK IMPORTING CO',
       'E & J GALLO WINERY', 'WILLIAM GRANT & SONS INC',
       'HEAVEN HILL DISTILLERIES', 'DISARONNO INTERNATIONAL LLC',
       'EDRINGTON AMERICAS', 'CASTLE BRANDS CORP.',
       'SOUTHERN WINE & SPIRITS NE', 'STE MICHELLE WINE ESTATES',
       'TRINCHERO FAMILY ESTATES', 'MHW LTD', 'WINE GROUP INC',
       'PERFECTA WINES', 'LUXCO INC', 'TREASURY WINE ESTATES',
       'DIAGEO CHATEAU ESTATE WINES', 'SHAW ROSS INT L IMP LTD',
       'PINE STATE TRADING CO', 'PHILLIPS PRO

In [29]:
vendor_sale_summery["GrossProfit"]=vendor_sale_summery["Total_Sale_Dollars"]-vendor_sale_summery["Total_Purchase_Dollars"]

In [31]:
vendor_sale_summery["ProfitMargin"]=(vendor_sale_summery["GrossProfit"]/vendor_sale_summery["Total_Sale_Dollars"])*100

In [37]:
vendor_sale_summery["StockTurnover"]=vendor_sale_summery["Total_Sale_Quantity"]/vendor_sale_summery["Total_Purchase_Quantity"]

In [38]:
vendor_sale_summery["SalestoPurchaseRatio"]=vendor_sale_summery["Total_Sale_Dollars"]/vendor_sale_summery["Total_Purchase_Dollars"]

In [40]:
vendor_sale_summery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   VendorNumber             10692 non-null  int64  
 1   VendorName               10692 non-null  object 
 2   Brand                    10692 non-null  int64  
 3   Description              10692 non-null  object 
 4   ActualPrice              10692 non-null  float64
 5   PurchasePrice            10692 non-null  float64
 6   Volume                   10692 non-null  float64
 7   Total_Purchase_Quantity  10692 non-null  int64  
 8   Total_Purchase_Dollars   10692 non-null  float64
 9   Total_Sale_Quantity      10692 non-null  float64
 10  Total_Sale_Dollars       10692 non-null  float64
 11  Total_Sale_Price         10692 non-null  float64
 12  Total_Excise_Tax         10692 non-null  float64
 13  Freight_Cost             10692 non-null  float64
 14  GrossProfit           

In [39]:
cursor=conn.cursor()

In [41]:
cursor.execute("""
create table vendor_sale_summery(
VendorNumber INT,
VendorName VARCHAR(100), 
Brand INT,
Description VARCHAR(100),
PurchasePrice DECIMAL(10,2),
ActualPrice DECIMAL(10,2),
Volume INT,
Total_Purchase_Quantity INT,
Total_Purchase_Dollars DECIMAL(15,2),
Total_Sale_Quantity INT,
Total_Sale_Dollars DECIMAL(15,2),
Total_Sale_Price DECIMAL(15,2),
Total_Excise_Tax DECIMAL(15,2),
Freight_Cost DECIMAL(15,2),
GrossProfit DECIMAL(15,2),
ProfitMargin DECIMAL(15,2),
StockTurnover DECIMAL(15,2),
SalestoPurchaseRatio DECIMAL(15,2),
PRIMARY KEY (VendorNumber, Brand)
);
""")

In [42]:
pd.read_sql_query("select * from vendor_sale_summery",conn)

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,Total_Purchase_Quantity,Total_Purchase_Dollars,Total_Sale_Quantity,Total_Sale_Dollars,Total_Sale_Price,Total_Excise_Tax,Freight_Cost,GrossProfit,ProfitMargin,StockTurnover,SalestoPurchaseRatio


In [44]:
vendor_sale_summery.to_sql("vendor_sale_summery",conn,if_exists="replace",index=False)

10692

In [46]:
pd.read_sql_query("select * from vendor_sale_summery",conn)

,VendorNumber,VendorName,Brand,Description,ActualPrice,PurchasePrice,Volume,Total_Purchase_Quantity,Total_Purchase_Dollars,Total_Sale_Quantity,Total_Sale_Dollars,Total_Sale_Price,Total_Excise_Tax,Freight_Cost,GrossProfit,ProfitMargin,StockTurnover,SalestoPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,36.99,26.27,1750.0,145080,3811251.60,142049.0,5.101920e+06,672819.31,260999.20,68601.68,1290667.91,25.297693,0.979108,1.338647
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,28.99,23.19,1750.0,164038,3804041.22,160247.0,4.819073e+06,561512.37,294438.66,144929.24,1015032.27,21.062810,0.976890,1.266830
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,24.99,18.24,1750.0,187407,3418303.68,187140.0,4.538121e+06,461140.15,343854.07,123780.22,1119816.92,24.675786,0.998575,1.327594
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,22.99,16.17,1750.0,201682,3261197.94,200412.0,4.475973e+06,420050.01,368242.80,257032.07,1214774.94,27.139908,0.993703,1.372493
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,29.99,21.89,1750.0,138109,3023206.01,135838.0,4.223108e+06,545778.28,249587.83,257032.07,1199901.61,28.412764,0.983556,1.396897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,4.99,1.32,750.0,2,2.64,5.0,1.595000e+01,10.96,0.55,27100.41,13.31,83.448276,2.500000,6.041667
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.49,0.39,50.0,6,2.34,134.0,6.566000e+01,1.47,7.04,50293.62,63.32,96.436186,22.333333,28.059829
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.99,0.74,50.0,2,1.48,2.0,1.980000e+00,0.99,0.10,14069.87,0.50,25.252525,1.000000,1.337838
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.99,1.47,200.0,1,1.47,72.0,1.432800e+02,77.61,15.12,257032.07,141.81,98.974037,72.000000,97.469388
